In [0]:
pip install jsonpickle

In [0]:
import numpy as np
from hmmlearn.base import ConvergenceMonitor
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
import os
import json
import jsonpickle
import hmmlearn

#HMM ON GPS DATA

In [0]:
#from dataset take 80% for training and 20% testing (for ex)

def dataa():
    tr_data=80 #int(input("enter percentage of training data:    ")) #in percentage
    #print("training data % is ",tr_data)
    pr_data=100-tr_data #in percentage
    #print("prediction data % is: ",(pr_data))
    return(tr_data,pr_data)
    
    


In [0]:
def datab(file): #splitting dataset into training and testing data
    #tr=0
    with open(file, 'r+') as f:
        lines=f.read().split('\n')
        a=(len(lines))
        f.close()
    p100 = dataa()
    tr_data = p100[0]
    pr_data = p100[1]
    #print("total number of data points = ",a)
    tr=int(round(a*((tr_data)/100) ))
    #print("Number of training data points taken =" ,tr)
    pr=a-tr
    #print("Number of testing data points taken = ",pr)
    spli=train_test_split(lines,train_size=tr,test_size=pr,shuffle=False)
    return (tr,pr,spli,p100)
    


In [0]:
def nmser(x,y):  #to find normalized mean square error-NMSE
    z=0 
    if len(x)==len(y):
        for k in range(len(x)):
            z=z+(((float(x[k])-y[k])**2)/float(x[k]))    
            
    elif len(x)>>len(y) or len(x)<<len(y):
        print('invalid list length: please check the list')
    
    z=z/(len(x))
    return z 

In [0]:

def writeToJSONFile(path, fileName, data):
    filePathNameWExt = '' + path + '/' + fileName + '.txt'
    with open(filePathNameWExt, 'a') as fp :
        json.dump(data, fp)


In [0]:
#runs throughout the directory and appends list with .plt files address
def findFilesInFolder(path, pathList, extension, subFolders = True):
    import os
    
 
    try:   # error might be du to File permissions 
        with os.scandir(path) as it:
            for entry in it:
                if entry.is_file() and entry.path.endswith(extension):
                    pathList.append(entry.path)
                    
                                  
                elif entry.is_dir() and subFolders:   # if its a directory(folder), then repeat process as a nested function
                    pathList = findFilesInFolder(entry.path, pathList, extension, subFolders)
                
    except OSError:
        print('Cannot access ' + path +'. Probably a permissions error')
        
    return pathList


In [0]:
class Thing(object): #to convert hmmlearn into json 
    def __init__(self, name):
        self.name = name

In [0]:
def fileexistsdelete(path,fileName): #if text file exists then delete and then write result
    filePathNameWExt = '' + path + '/' + fileName + '.txt'
    try:
        os.remove(filePathNameWExt)
    except OSError:
        pass

In [0]:

def model_prog(path, pathList, extension, subFolders):
    #path=model_prog1(path, pathList, extension, subFolders)
    p1=os.scandir(path)
    #print(p1)
    for k in p1:
       
        path1=os.path.join(k)
        #path1=os.path.dirname(os.path.abspath(k))
        #print(k)
        pathList=findFilesInFolder(path1, pathList, extension, subFolders)
       
        fileName=os.path.basename(path1)
        
        fileexistsdelete(path1,fileName)
        
        for count,j in enumerate(pathList):
            #print(count)
            #print(j)
            p22=datab(j)
            spli=p22[2]
            tr=p22[0]
            pr=p22[1]
            train=(spli[0][0:])
            test=(spli[1][0:])
            
            obs = np.array([line.rstrip('\n') for line in train])
            
            X=np.reshape(obs,(-1,1))
            
            X=X.astype(np.float64)
            #print(len(X))
            
            model = (hmm.GaussianHMM(n_components=2,covariance_type="full",algorithm='viterbi',n_iter=10, random_state=7))
            #modela=hmmlearn.stats(model)#.fit(X))
            #print(modela)
            mod = jsonpickle.encode(Thing(model.fit(X)))
            
            model.monitor_
            ConvergenceMonitor(n_iter=10,    tol=0.01,    verbose=False)
            
            model.monitor_.converged
            
            y_pr=model.sample(pr)
            
            
            p = [x[0] for x in y_pr[0]]        
            z=nmser(test,p)
            
            file=os.path.basename(j)
            l1=[]
           
            l1.append({'COUNT: ':count,
                       'Dataset':file,
                       'model':mod, 
                       'data':p22,
                       'predicted data':p,
                       'NMSE':z})
            
            a=writeToJSONFile(path1,fileName,l1)
        
        l1.clear()
        pathList.clear()
        del pr,X,p22,p,z,l1,model,j

In [0]:
path=('I:/VIT/data ericsson/data/test')#,encoding='utf-8')

model_prog(path, [], '.plt',True)

FileNotFoundError: ignored